## Performance of the CPL
Author: Clara Hoffmann \
Measures the performance of the CPL on the precise validation set.

In [15]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy as sp
from tqdm import tqdm
from scipy.integrate import simps, trapz
import matplotlib.pyplot as plt
import imageio
import multiprocessing
from helpers import (Fy, find_closest_element,  compute_coverage, #predict_single_density,
confidence_interval, confidence_interval, generate_fixed_terms, get_ci)
from density_predictor_class import density_predictor

In [16]:
# read in val data
B_zeta = np.load('../../../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/Bzeta/B_zeta_val.npy')
B_zeta = B_zeta.reshape(B_zeta.shape[0], B_zeta.shape[2])
true_y = np.load('../../../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/Bzeta/labels_val.npy')

density_path = '../../../../data/commaai/density/gaussian_density_filtered.csv'
density = pd.read_csv(density_path)

# number of points at which to evaluate the predictive densities
no_points = 500

In [19]:
# mc_dropout predictions:
preds = np.append(np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_1.npy'), 
                  np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_1.npy'), axis = 0)
preds_mc = np.mean(preds, axis = 1)

In [20]:
dens = density_predictor(B_zeta, true_y, density, no_points)

In [21]:
result_va_ridge = dens.get_density('va_ridge')
result_hmc_ridge = dens.get_density('hmc_ridge')
result_va_horse = dens.get_density('va_horseshoe')
result_hmc_horse = dens.get_density('hmc_horseshoe')

  1%|          | 73/10472 [00:00<00:14, 727.67it/s]

computing densities for each observation


  4%|▎         | 386/10472 [00:00<00:02, 3855.44it/s]

computing mean prediction for each observation


 10%|█         | 1052/10472 [00:00<00:00, 10514.29it/s]

computing variance prediction for each observation


 15%|█▍        | 1562/10472 [03:22<19:14,  7.72it/s]


KeyboardInterrupt: 

In [ ]:
print('Validation Performance: ' +
      '\nMSE:' +
      '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0)**2)) +
     '\n CPL VA/Ridge: ' + str(np.mean((true_y - np.array(result_va_ridge['mean predictions']))**2)) +
     '\n CPL HMC/Ridge: ' + str(np.mean((true_y - np.array(result_hmc_ridge['mean prediction']))**2)) +
     '\n CPL VA/Horseshoe: ' + str(np.mean((true_y - np.array(result_va_horse['mean prediction']))**2)) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean((true_y - np.array(result_hmc_horse['mean prediction']))**2)) +
      '\n MC-Dropout: ' + str(np.mean((true_y - preds_mc)**2)) +
      
    '\nMAE:' +
      '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0))) +
     '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])))) +
     '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])))) +
     '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])))) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])))) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc))) +
      
      '\nAccuracy:' +
     '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0) < 6)) +
     '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) < 6)) +
     '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) < 6)) +
     '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) < 6)) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])) < 6)) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc < 6))
      
      '\nAccuracy II :' +
      '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0) < 6)) +
     '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) < 2)) +
     '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) < 2)) +
     '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) < 2)) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])) < 2)) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc) < 2))
      
     )

In [ ]:
df = pd.DataFrame({'va_ridge' : result_va_ridge['densities'],
                 'hmc_ridge' : result_hmc_ridge['densities'],
                 'va_horse' : result_va_horse['densities'],
                 'hmc_horse' : result_hmc_horse['densities']})

In [ ]:
df.to_csv('../../../../data/commaai/predictions/cpl/densities.csv')